In [69]:
import pandas as pd
import requests
import time
import re
import numpy as np
from bs4 import BeautifulSoup
from sklearn.linear_model import LinearRegression

In [88]:
data = pd.DataFrame(columns= ['Name', 'Year','Wins','Champion','MVP','Scoring Leader', 
                              'Rebound Leader', 'Assist Leader', 'WS Leader','DPOY',
                              'MIP', '6MOY', 'Coach of Year', 'All-Stars', 'All-Defensive',
                              #'All-NBA', FG%', '3P%', 
                              #'2P%', 'FT%', 'PPG', 'TOV', 'AST', 'STL', 'BLK'  
                             ])
data

,Name,Year,Wins,Champion,MVP,Scoring Leader,Rebound Leader,Assist Leader,WS Leader,DPOY,MIP,6MOY,Coach of Year,All-Stars,All-Defensive


In [89]:
def check_roster(team_soup, season_awards, allst_roster, all_def):
    awards = [0,0,0,0,0,0,0,0,0,0,0]
    players = team_soup.find_all('td', {'class':'left', 'data-stat':'player'})
    roster = [player.text for player in players]
    coach = roster.append(team_soup.find(text=re.compile('Coach:')).parent.parent.find('a').text)
    
    star_count = 0
    all_def_count = 0
    for allst in allst_roster:
        if allst in roster:
            star_count += 1
 
    awards[9] = star_count
    for i in range(9):
        if season_awards[i] in roster:
            awards[i] = 1
            
    for player in roster:
        if player in all_def:
            all_def_count +=1 
    awards[10] = all_def_count
    
    return awards

In [90]:
def get_team_row(all_def, season_awards, allst_roster, name, team_url, year, wins):
    row = []
    row.append(name.replace('*',''))
    row.append(year)
    row.append(wins)
    
    roster = []
    
    url = 'http://www.basketball-reference.com' + team_url
    req = requests.get(url)
    time.sleep(0.1)
    team_soup = BeautifulSoup(req.text, 'html.parser')
    try:
        row.append(team_soup.find('br').text[:4] == 'Won ')
    except:
        row.append(False)
    row += check_roster(team_soup, season_awards, allst_roster,all_def)
    
    row += get_statline(team_soup)
    
    return row

In [91]:
leaders = ['Most Valuable Player', 'PPG Leader', 'RPG Leader',
           'APG Leader', 'WS Leader']

awards = ['Defensive Player of the Year', 'Most Improved Player', 
          'Sixth Man of the Year', 'Coach of the Year']
winners = []

def get_award_winners(season, year):
    for leader in leaders:
        winners.append(season.body.find(text=leader).parent.parent.find('a').text)

    #Using a different site that's more convenient to scrape other awards.
    year_string = '%s' % str(year)
    next_year = '%s' % str(year+1)
    year = '%s-%s' % (year_string, next_year[2:4])
    for award in awards:
        url = 'http://www.nba.com/history/%s-award/' %(award.lower().replace(' ', '-'))
        req = requests.get(url)
        soup = BeautifulSoup(req.text, 'html.parser')

        table = soup.find('table')    
        try:
            winners.append(table.find(text=re.compile(year)).parent.parent.find_all('td')[1].text.strip())
        except:
            winners.append('')
    
    return winners 

In [92]:
# Append all_stars rosters to list
def get_allstars(year):
    url = "http://www.basketball-reference.com/allstar/NBA_%s.html/" % str(year)
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')
    all_stars = []
    
    for row in soup.find("div", {"id" : "all_East"}).find_all("th", {"class" : "left", "scope" : "row", "csk" : True}):
        all_stars.append(row.find("a").text)
        
        
    for row in soup.find("div", {"id" : "all_West"}).find_all("th", {"class" : "left", "scope" : "row", "csk" : True}):
        all_stars.append(row.find("a").text)
        
    return all_stars

In [93]:
## Because we need only 1 request to get the whole defensive page, each five players from list is from the First team of a season.
def get_all_def():
    url = "http://www.nba.com/history/all-defensive-teams/"
    req = requests.get(url)
    defensive = BeautifulSoup(req.text, 'html.parser')
    all_defense = []
    
    for i in defensive.find_all("td" , {"class" : "cnnIEColTxtL", "style" : "text-align: center"}):
        if i.text == "First Team":
            continue
        all_defense.append(i.text)
        
    for i in defensive.find_all("td" , {"class" : "cnnIEColTxtR", "style" : "text-align: center"}):
        if i.text == "Second Team":
            continue
        all_defense.append(i.text)
    return all_defense

In [94]:
def get_statline(team_soup):
        
    fg_pct = team_soup.find('tr', {'data-row':'0'})
    print(fg_pct)

In [95]:
row_num = 0
all_def = get_all_def()

for i in range(1975, 1976):
    url = ('http://www.basketball-reference.com/leagues/NBA_%s.html' % str(i))
    time.sleep(0.1)
    req = requests.get(url)
    season = BeautifulSoup(req.text, 'html.parser')
    teams = season.find_all('th', {'class': 'left', 'data-stat':'team_name'})
   
    season_awards = get_award_winners(season, i-1)
    
    allst_roster = get_allstars(i)
    
    for team in teams:
        if(team.text == 'Team' or 'Conference' in team.text):
            continue
        wins = int(team.parent.find('td', {'data-stat':'wins'}).text)
     
        team_name = repr(team.text).replace('\\x','*').split('*')[0].lstrip("'")
        for a in team.find_all('a',href=True):
            new_row = get_team_row(all_def,season_awards,allst_roster,team_name,a['href'], i, wins)
  
            data.loc[row_num] = new_row
            row_num += 1


None


TypeError: 'NoneType' object is not iterable

In [ ]:
data

In [96]:
req = requests.get('http://www.basketball-reference.com/teams/BOS/2006.html')
soup = BeautifulSoup(req.text, 'html.parser')

In [97]:
data_rows = soup.findAll('tr')[2:]  # skip the first 2 header rows

In [99]:
data_rows[3]

<tr><th class="center " data-stat="number" scope="row">20</th><td class="left " csk="Dickau,Dan" data-append-csv="dickada01" data-stat="player"><a href="/players/d/dickada01.html">Dan Dickau</a></td><td class="center " csk="1" data-stat="pos">PG</td><td class="right " csk="72.0" data-stat="height">6-0</td><td class="right " data-stat="weight">190</td><td class="left " csk="19780916" data-stat="birth_date">September 16, 1978</td><td class="right " data-stat="birth_country"><span class="f-i f-us">us</span></td><td class="right " csk="3" data-stat="years_experience">3</td><td class="left " data-stat="college_name"><a href="/friv/colleges.fcgi?college=gonzaga">Gonzaga University</a></td></tr>

In [100]:
player_data = [[td.getText() for td in data_rows[i].findAll('td')]
               for i in range(len(data_rows))]

In [102]:
player_data[0]

['Marcus Banks',
 'PG',
 '6-2',
 '200',
 'November 19, 1981',
 'us',
 '2',
 'University of Nevada, Las Vegas']

In [103]:
for i in range(len(data_rows)):
    [td.getText() for td in data_rows[i].findAll('td')]   

In [104]:
player_data_02 = []  # create an empty list to hold all the data

for i in range(len(data_rows)):  # for each table row
    player_row = []  # create an empty list for each pick/player

    # for each table data element from each table row
    for td in data_rows[i].findAll('td'):        
        # get the text content and append to the player_row 
        player_row.append(td.getText())        

    # then append each pick/player to the player_data matrix
    player_data_02.append(player_row)

In [105]:
player_data_02

[['Marcus Banks',
  'PG',
  '6-2',
  '200',
  'November 19, 1981',
  'us',
  '2',
  'University of Nevada, Las Vegas'],
 ['Mark Blount',
  'C',
  '7-0',
  '230',
  'November 30, 1975',
  'us',
  '5',
  'University of Pittsburgh'],
 ['Ricky Davis',
  'SG',
  '6-6',
  '195',
  'September 23, 1979',
  'us',
  '7',
  'University of Iowa'],
 ['Dan Dickau',
  'PG',
  '6-0',
  '190',
  'September 16, 1978',
  'us',
  '3',
  'Gonzaga University'],
 ['Ryan Gomes',
  'PF',
  '6-7',
  '250',
  'September 1, 1982',
  'us',
  'R',
  'Providence College'],
 ['Gerald Green', 'SG', '6-7', '205', 'January 26, 1986', 'us', 'R', ''],
 ['Orien Greene',
  'PG',
  '6-4',
  '208',
  'February 4, 1982',
  'us',
  'R',
  'University of Louisiana at Lafayette'],
 ['Al Jefferson', 'PF', '6-10', '289', 'January 4, 1985', 'us', '1', ''],
 ['Dwayne Jones',
  'PF',
  '6-11',
  '250',
  'June 9, 1983',
  'us',
  'R',
  "Saint Joseph's University"],
 ['Raef LaFrentz',
  'PF',
  '6-11',
  '240',
  'May 29, 1976',
  'us